In [82]:
from matching.algorithms import galeshapley
import operator
import random
from IPython.core.debugger import set_trace
import numpy as np

n = 400
sigma = 1

In [83]:
#make randomly assigned preferences
Mpref = {}
Wpref = {}
for i in range(0, n):
    utility = []
    for x in range(0,n):
        id = 'W' + str(x)
        utility = utility + [(id, random.normalvariate(x, sigma))]
    utility.sort(key=operator.itemgetter(1))
    
    ranklist = []
    for x in utility:
        name = [x[0]]
        ranklist = ranklist + name
    id = 'M' + str(i)
    Mpref.update({id:ranklist})
    
#copypasta from above
for i in range(0, n):
    utility = []
    for x in range(0,n):
        id = 'M' + str(x)
        utility = utility + [(id, random.normalvariate(x, sigma))]
    utility.sort(key=operator.itemgetter(1))
    
    ranklist = []
    for x in utility:
        name = [x[0]]
        ranklist = ranklist + name
    id = 'W' + str(i)
    Wpref.update({id:ranklist})
    

print(Mpref)
print(Wpref)

{'M0': ['W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W7', 'W6', 'W8', 'W9', 'W11', 'W10', 'W12', 'W14', 'W13', 'W15', 'W16', 'W18', 'W17', 'W20', 'W19', 'W21', 'W22', 'W25', 'W23', 'W24', 'W26', 'W28', 'W27', 'W30', 'W31', 'W29', 'W33', 'W32', 'W34', 'W35', 'W36', 'W38', 'W37', 'W40', 'W39', 'W42', 'W41', 'W43', 'W44', 'W45', 'W46', 'W47', 'W50', 'W48', 'W49', 'W52', 'W51', 'W53', 'W54', 'W55', 'W56', 'W59', 'W58', 'W57', 'W60', 'W61', 'W64', 'W63', 'W62', 'W66', 'W65', 'W67', 'W68', 'W69', 'W70', 'W71', 'W73', 'W72', 'W74', 'W75', 'W76', 'W77', 'W78', 'W79', 'W81', 'W80', 'W82', 'W83', 'W84', 'W87', 'W85', 'W86', 'W90', 'W88', 'W89', 'W91', 'W93', 'W92', 'W94', 'W96', 'W95', 'W97', 'W98', 'W99', 'W100', 'W101', 'W102', 'W104', 'W103', 'W105', 'W106', 'W107', 'W108', 'W109', 'W110', 'W111', 'W112', 'W114', 'W113', 'W116', 'W115', 'W118', 'W119', 'W117', 'W120', 'W121', 'W123', 'W124', 'W122', 'W125', 'W127', 'W126', 'W128', 'W130', 'W131', 'W129', 'W133', 'W132', 'W134', 'W135', 'W136', 'W138'

In [84]:
M_propose= galeshapley(Mpref, Wpref)
W_propose = galeshapley(Wpref, Mpref)
M_propose = list(M_propose.items())
W_propose = [(v, k) for k, v in W_propose.items()]


In [85]:
M_propose.sort(key=operator.itemgetter(0))
W_propose.sort(key=operator.itemgetter(0))
M_propose
W_propose

[('M0', 'W0'),
 ('M1', 'W1'),
 ('M10', 'W9'),
 ('M100', 'W101'),
 ('M101', 'W99'),
 ('M102', 'W103'),
 ('M103', 'W102'),
 ('M104', 'W105'),
 ('M105', 'W104'),
 ('M106', 'W106'),
 ('M107', 'W108'),
 ('M108', 'W107'),
 ('M109', 'W110'),
 ('M11', 'W10'),
 ('M110', 'W109'),
 ('M111', 'W112'),
 ('M112', 'W111'),
 ('M113', 'W116'),
 ('M114', 'W114'),
 ('M115', 'W115'),
 ('M116', 'W113'),
 ('M117', 'W118'),
 ('M118', 'W117'),
 ('M119', 'W120'),
 ('M12', 'W13'),
 ('M120', 'W119'),
 ('M121', 'W122'),
 ('M122', 'W121'),
 ('M123', 'W123'),
 ('M124', 'W125'),
 ('M125', 'W124'),
 ('M126', 'W127'),
 ('M127', 'W130'),
 ('M128', 'W126'),
 ('M129', 'W128'),
 ('M13', 'W12'),
 ('M130', 'W131'),
 ('M131', 'W129'),
 ('M132', 'W132'),
 ('M133', 'W133'),
 ('M134', 'W134'),
 ('M135', 'W135'),
 ('M136', 'W136'),
 ('M137', 'W137'),
 ('M138', 'W138'),
 ('M139', 'W140'),
 ('M14', 'W15'),
 ('M140', 'W139'),
 ('M141', 'W143'),
 ('M142', 'W142'),
 ('M143', 'W141'),
 ('M144', 'W144'),
 ('M145', 'W145'),
 ('M146', 'W1

In [86]:
c = (np.array(M_propose) == np.array(W_propose))
c

np.average(c, axis = 0)


array([1.    , 0.9575])